In [2]:

# core code for predicting the shear strength of RC deep beams
# v 1.0
# 2020-02-23

# define some necessary packages
import os
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt

from openpyxl import load_workbook
from sklearn.preprocessing import StandardScaler

from sklearn import linear_model
from sklearn import svm
from sklearn import tree
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import matplotlib
matplotlib.rcParams['mathtext.fontset']='stix'
import xgboost as xgb

from sklearn.inspection import permutation_importance

In [3]:
def MAPE(true, pred):
    diff = np.abs(np.array(true) - np.array(pred))
    return np.mean(diff / true)

In [5]:
# load the full data set
# dataset=np.loadtxt('DeepBeam2.csv',delimiter=",")
dataset = pd.read_excel('C:/Users/WjW/Desktop/2021.2.1/DeepBeam2.xlsx', sheet_name='DeepBeam2')
dataset.head()

,fc,b,h,a,h0,l0,a/h0,l0/h,sv,fyv,rv,sh,fyh,rh,fy,ry,V
0,21.45,100.0,750.0,500.0,690.11,1500.0,0.72,2.00,0.00,0.0,0.00,0.00,0.0,0.00,210.0,1.82,249.90
1,49.10,130.0,560.0,625.0,500.00,2000.0,1.25,3.57,362.82,410.0,0.12,101.25,410.0,0.43,410.0,1.56,347.10
2,23.70,130.0,560.0,425.0,500.00,2000.0,0.85,3.57,362.82,410.0,0.12,101.25,410.0,0.43,410.0,1.56,284.05
3,21.20,76.0,508.0,254.0,470.00,762.0,0.54,1.50,76.00,279.9,2.45,0.00,0.0,0.00,286.8,0.79,190.00
4,42.80,110.0,500.0,1250.0,462.96,2500.0,2.70,5.00,300.00,375.2,0.48,0.00,0.0,0.00,504.8,1.23,105.00


In [6]:
# reading the original input variables from the database
fc = dataset.loc[:, 'fc']
b = dataset.loc[:, 'b']
h = dataset.loc[:, 'h']
a = dataset.loc[:, 'a']
h0 = dataset.loc[:, 'h0']
l0 = dataset.loc[:, 'l0']
sv = dataset.loc[:, 'sv']
fyv = dataset.loc[:, 'fyv']
rv = dataset.loc[:, 'rv']
sh = dataset.loc[:, 'sh']
fyh = dataset.loc[:, 'fyh']
rh = dataset.loc[:, 'rh']
fy = dataset.loc[:, 'fy']
ry = dataset.loc[:, 'ry']
V = dataset.loc[:, 'V']



In [10]:
# constructing 6 new normalized dimensionless input variables
X = np.zeros(shape=(272,16))
X[:, 0] = fc
X[:, 1] = b
X[:, 2] = h
X[:, 3] = a
X[:, 4] = h0
X[:, 5] = l0
X[:, 6] = a/h0
X[:, 7] = l0/h
X[:, 8] = sv
X[:, 9] = fyv
X[:, 10] = rv
X[:, 11] = sh
X[:, 12] = fyh
X[:, 13] = rh
X[:, 14] = fy
X[:, 15] = ry
print(X.shape)
print(X)

(272, 16)
[[2.145e+01 1.000e+02 7.500e+02 ... 0.000e+00 2.100e+02 1.820e+00]
 [4.910e+01 1.300e+02 5.600e+02 ... 4.300e-01 4.100e+02 1.560e+00]
 [2.370e+01 1.300e+02 5.600e+02 ... 4.300e-01 4.100e+02 1.560e+00]
 ...
 [5.120e+01 1.100e+02 5.000e+02 ... 0.000e+00 5.048e+02 1.230e+00]
 [2.757e+01 1.000e+02 9.000e+02 ... 3.500e-01 3.340e+02 2.700e-01]
 [4.910e+01 1.300e+02 5.600e+02 ... 4.300e-01 4.100e+02 1.560e+00]]


In [11]:
# defining the output, i.e., the shear strength of the corroded beam
y = V

print(y.shape)

(272,)


In [12]:
# normalize the data sets
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

In [14]:
# split the training-testing set into 10 folds
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=10)

# define training algorithms
# RF
regr_1 = RandomForestRegressor(n_estimators=80,max_depth=8,max_leaf_nodes=250,min_samples_leaf=2,min_samples_split=5,bootstrap= True,random_state=0,max_features=12)

# # GBRT
regr_2 = GradientBoostingRegressor(n_estimators=500, learning_rate=0.02,max_depth=8,min_samples_leaf=2,min_samples_split=2, random_state=0, loss='ls')

# # AdaBoost
regr_3 = AdaBoostRegressor (tree.DecisionTreeRegressor(max_depth=8,min_samples_split=3,min_samples_leaf=2,max_leaf_nodes=15,criterion='mse'),n_estimators=400, learning_rate=0.1, random_state=0)

# XGB
regr_4 = xgb.XGBRegressor (max_depth=8, learning_rate=0.02, n_estimators=600, colsample_bytree=1, subsample=0.69,  gamma=0, random_state=0)

#DT
regr_5 = DecisionTreeRegressor(max_depth=20,max_leaf_nodes=80,min_samples_leaf=2,min_samples_split=12,random_state=0)

# #SVM
regr_6 = svm.SVR(kernel='rbf', C=1500, gamma=0.04,coef0=0)

#ANN
# regr_7 = MLPRegressor(hidden_layer_sizes=(5,), solver='lbfgs', alpha=0.001, random_state=0, epsilon=1e-08)
regr_7 = MLPRegressor(hidden_layer_sizes=(8,), solver='lbfgs', random_state=0, max_iter=500)
###
regr = regr_3
predicted = cross_val_predict (regr, X_train, y_train, cv=10)
scores = cross_val_score(regr, X_train, y_train, cv=10, scoring='r2', n_jobs = -1)
print(scores.mean())

0.8323957470901249


In [15]:
# validate the predictive model
regr.fit(X_train, y_train)

Z = regr.predict(X_train)
Z1 = regr.predict(X_test)

print(" Training RMSE:", np.sqrt(mean_squared_error(y_train, Z)), "MAE:", mean_absolute_error(y_train, Z), "MAPE:", MAPE(y_train, Z), "R2:", r2_score(y_train, Z))
print(" Testing RMSE:", np.sqrt(mean_squared_error(y_test, Z1)), "MAE:", mean_absolute_error(y_test, Z1), "MAPE:", MAPE(y_test, Z1), "R2:", r2_score(y_test, Z1))


 Training RMSE: 29.949584429469926 MAE: 25.089927262309313 MAPE: 0.12057827176551532 R2: 0.971616560270381
 Testing RMSE: 52.241340551507385 MAE: 35.653125374667205 MAPE: 0.12718930830114877 R2: 0.9225905083293233


In [ ]:
################## -------------------- step 1
### plot the results 
# xx = np.linspace(0, 1400, 100)
# yy = xx

# plt.figure()
# plt.plot(xx, yy, c='k', linewidth=2)
# plt.scatter(y_train, Z, marker='s')
# plt.scatter(y_test, Z1, marker='o')
# plt.grid()

# plt.tick_params (axis='both',which='major',labelsize=14)
# plt.yticks(size = 14)
# plt.xticks(size = 14)
# font1 = {'family' : 'sans-serif', 'weight' : 'normal', 'size' :16,}
# plt.axis('tight')
# plt.xlabel('Tested shear strength (kN)', font1)
# plt.ylabel('Predicted shear strength (kN)', font1)
# plt.xlim([-100, 1500])
# plt.ylim([-100, 1500])
# plt.legend(['y=x','Training set','Testing set'], loc = 'upper left', fontsize=14)
# plt.savefig('Fig6d.eps', dpi=600, bbox_inches = 'tight', format='eps')
# plt.show()

In [ ]:
################## -------------------- step 2
### plot the accuracy of ML models

# y_pre = regr.predict(X)
# y_ture = dataset[0:, 16]

# ratio = y_ture/y_pre

# # plot the accuracy of experience models
# dataset1=np.loadtxt('steel.csv',delimiter=",")
# # ratio = dataset1[0:, 6]#GB
# ratio = dataset1[0:, 7]#ACI
# # ratio = dataset1[0:, 8]#CSA
# # ratio = dataset1[0:, 9]#EC2EC2


# mean = np.mean(ratio)
# std = np.std(ratio, ddof=1)
# xx = np.linspace(0, 3, 3)
# yy = mean*np.ones(3)
# yy1 = (mean+std)*np.ones(3)
# yy2 = (mean-std)*np.ones(3)

# plt.figure()
# plt.plot(xx, yy, c='k', linewidth=2)
# plt.plot(xx, yy1, c='b', ls='--', linewidth=2)
# plt.plot(xx, yy2, c='b', ls='--', linewidth=2)

# # sample = dataset[0:, 6]
# sample = dataset1[0:, 0]
# # sample = np.arange(1, len(ada_ratio)+1)dataset[0:, 0:16]
# plt.scatter(sample, ratio, c='r', marker='o')

# plt.grid()
# plt.tick_params(axis='both',which='major',labelsize=14)
# font1 = {'family' : 'sans-serif', 'weight' : 'normal', 'size' : 16,}
# plt.axis('tight')
# plt.xlabel('a/d ratio', font1)
# plt.ylabel('Predict-to-test ratio', font1)
# plt.ylim([0, 4])
# plt.legend(['Mean','Mean+St.D.','Mean-St.D.','XGBoost'], loc = 'upper right', fontsize=12)


# print(' mean :', mean, ' std :', std, ' COV :', std/mean)
# plt.show()



In [ ]:
################## -------------------- step 3
### plot the accuracy of ML models

# data = pd.read_csv('DeepBeam2 - fea.csv')
# x=[r'fc',r'b',r'h',r'a',r'h0',r'l0',r'a/h0',r'l0/h',r'sv',r'fyv',r'ρv',r'sh',r'fyh',r'ρh',r'fy',r'ρy']
# X = data[x]
# y = data['V']

# scaler = StandardScaler()
# scaler.fit(X)
# X = scaler.transform(X)

# # train XGBoost model
# model= xgb.XGBRegressor (max_depth=8, learning_rate=0.02, n_estimators=600, colsample_bytree=1, subsample=0.69,  gamma=0, random_state=0)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

# # 10-fold cv results for hyper-parameter validation
# scores = cross_val_score(model, X_train, y_train, cv=10, scoring='r2', n_jobs = -1)
# print(scores.mean())
# model.fit(X_train, y_train)

# xgb.plot_importance(model, max_num_features=16, importance_type='weight', height=0.6)

# # plot the feature imprtance
# feature_names = [r'$f_c$',r'$b$',r'$h$',r'$a$',r'$h_0$',r'$l_0$',r'$a/h_0$',r'$l_0/h$',r'$s_v$',r'$f_{yv}$',r'$\rho_v$',r'$s_h$',r'$f_{yh}$',r'$\rho_h$',r'$f_y$',r'$\rho_l$'] # 根据输入改

# plt.figure()

# model.importance_type = 'weight'

# feature_importance = model.feature_importances_

# # feature_importance = model.get_booster().get_score(importance_type='weight')

# # perform permutation importance
# # results = permutation_importance(model, X, y, scoring='neg_mean_squared_error')
# # get importance
# # feature_importance = results.importances_mean

# print(feature_importance)

# feature_importance = 100.0 * (feature_importance / feature_importance.max())
# sorted_idx = np.argsort(feature_importance)
# pos = np.arange(sorted_idx.shape[0]) + .5
# plt.barh (pos, feature_importance[sorted_idx], 0.6, align='center',)
# plt.yticks(pos, np.array(feature_names)[sorted_idx],style='italic')
# plt.yticks(rotation=360)

# for x, y in enumerate(feature_importance[sorted_idx]):
# 	plt.text(y + 0.5, x + 0.3, '%s' % '{:.1%}'.format(y/100))

# font1 = {'family' : 'sans-serif', 'weight' : 'normal', 'size' : 14,}
# plt.axis('tight')

# plt.tick_params(axis='both',which='major',labelsize=14)
# plt.yticks(size = 14)
# plt.xticks(size = 14)
# plt.xlabel('Relative importance ($\%$)', font1)
# plt.ylabel('Features', font1)
# plt.title('Feature importance', font1)
# # plt.xlim([0, 110])

# plt.grid()

# plt.savefig('Fig6.eps', dpi=600, bbox_inches = 'tight', format='eps')
# # plt.tight_layout(pad=3, w_pad=1.0, h_pad=1.0)
# plt.show()



In [ ]:
################## -------------------- step 4
### plot the partial dependence of ML models

# def plot_curve(X, Y, x_range, x_size):
#     #X represents fc values, Y represents predicted PHL ratios 
#     #x_range=[min,max],xsize represents the interval; exp.xlim=[20,180] xsize=20

#     xmin, xmax = x_range

#     plx = np.linspace(xmin+x_size/2, xmax-x_size/2, (xmax-xmin-x_size)/x_size+1)
#     plm = np.zeros(len(plx))
#     pls = np.zeros(len(plx))

#     for i in range(len(plx)):
#         temp=[]
#         for j in range(len(X)):        
#             if X[j] >= plx[i]-x_size/2 and X[j] < plx[i]+x_size/2:
#                 temp.append(Y[j])
#         if len(temp) == 0:
#             plm[i]=plm[i-1]
#             pls[i]=pls[i-1]
#         elif len(temp) == 1:
#             plm[i]=temp[0]
#             pls[i]=pls[i-1]
#         else:
#             plm[i] = np.mean(temp)
#             pls[i] = np.std(temp,ddof=1)

#     plt.figure()
#     plt.plot(plx,plm,'o-',color='r',label="Mean value")
#     plt.fill_between(plx,plm-pls,plm+pls,alpha=0.1,color='r')
#     plt.tick_params(axis = 'both', which = 'major', labelsize = 16)
#     plt.yticks(fontproperties = 'Times New Roman', size = 16)
#     plt.xticks(fontproperties = 'Times New Roman', size = 16)
#     font1 = {'family' : 'times new roman', 'weight' : 'normal', 'size' : 18,}
#     font2 = {'family' : 'times new roman', 'weight' : 'normal', 'size' : 14}

#     plt.axis('tight')
#     plt.ylabel('Ground truth/predited ratio',font1)
#     plt.xlim(x_range)
#     plt.ylim([0., 1.4])
#     plt.legend(loc = 'lower right', fontsize=14, prop=font2)

# #example  
# plot_curve(X_inp[:,4], ada_ratio, [20,180], 25)
# plt.xlabel('$f_c$ (MPa)',font1)
# plt.savefig('Fig12a.pdf', dpi=600, bbox_inches = 'tight', format='pdf')

# plot_curve(X_inp[:,11], ada_ratio, [0,0.8], 0.1)
# plt.xlabel('$n$',font1)
# plt.savefig('Fig12b.pdf', dpi=600, bbox_inches = 'tight', format='pdf')

# plot_curve(X_inp[:,6], ada_ratio, [0.5,6.5], 0.7)
# plt.xlabel(r'$\rho_s$ (\%)',font1)
# plt.savefig('Fig12c.pdf', dpi=600, bbox_inches = 'tight', format='pdf')

# plot_curve(X_inp[:,7], ada_ratio, [320,580], 50)
# plt.xlabel('$f_y$ (MPa)',font1)
# plt.savefig('Fig12d.pdf', dpi=600, bbox_inches = 'tight', format='pdf')

# # end
# plt.show()


# data = pd.read_csv('DeepBeam2 - fea.csv')
# data.info()

# X = data.loc[:, data.columns != 'V']
# y = data['V']

# x = ['fc','b','h','a','h0','l0','a/h0','l0/h','sv','fyv','ρv','sh','fyh','ρh','fy','ρy']
# # x = ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15']
# # X = data[x]
# # y = data['V']

# scaler = StandardScaler()
# scaler.fit(X)
# X = scaler.transform(X)

# # train XGBoost model
# model = xgb.XGBRegressor(max_depth=8, learning_rate=0.02, n_estimators=600, colsample_bytree=1, subsample=0.69,  gamma=0, random_state=0)

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

# model.fit(X_train, y_train)

# Z = model.predict(X_train)
# Z1 = model.predict(X_test)

# print(" Training RMSE:", np.sqrt(mean_squared_error(y_train, Z)), "MAE:", mean_absolute_error(y_train, Z), "MAPE:", MAPE(y_train, Z), "R2:", r2_score(y_train, Z))
# print(" Testing RMSE:", np.sqrt(mean_squared_error(y_test, Z1)), "MAE:", mean_absolute_error(y_test, Z1), "MAPE:", MAPE(y_test, Z1), "R2:", r2_score(y_test, Z1))

# # plot_partial_dependence (model, X_train, features=x, feature_names=x,
# #                         n_jobs=3, grid_resolution=10)


# print(x)
# data.head()



# from pdpbox import pdp


# # feature = f1
# pdp_goals = pdp.pdp_isolate (model = model, dataset = data, model_features = x, feature = 'b')

# pdp.pdp_plot(pdp_goals, 'b')

# plt.show()